In [38]:
import pandas as pd
import time
import pymysql

from tqdm import tqdm
from clickhouse_driver import Client
from calendar import monthrange

tqdm.pandas()

pd.options.mode.chained_assignment = None

start_time = time.time()

path = r'D:\Отчеты\категории_номеров\files\\'

client_click = Client(host='192.168.1.99', port='9000', user='default', password='jdfwl6812hwe',
                database='suitecrm_robot_ch', settings={'use_numpy': True})

for req_year in range(2017, 2024):
    for req_month in range(1, 13):
        time_1 = time.time()
        
        calls_month = (req_month - 3) % 12 + 1
        if calls_month > req_month:
            calls_year = req_year - 1
        else:
            calls_year = req_year
#         print(req_month, req_year, calls_month, calls_year)

        sql = f"""with temp_robot as (select *
                    from (select toDate(call_date)                                                      as call_date,
                                 uniqueid,
                                 substring(dialog, 11, 4)                                               as ochered,
                                 phone,
                                 row_number() over (partition by phone order by toDate(call_date) desc) as num
                          from suitecrm_robot_ch.jc_robot_log
                          where toDate(call_date) between toDate('{req_year}-{req_month}-01') - interval 2 month
                                    and toDate('{req_year}-{req_month}-01') + interval 1 month - interval 1 day) as temp
                    where temp.num = 1),
                     temp_requests as (select *
                                       from suitecrm_robot_ch.all_requests
                                       where toMonth(request_date) = {req_month}
                                         and toYear(request_date) = {req_year})

                select temp_requests.request_date,
                       temp_requests.project,
                       temp_requests.phone_request,
                       temp_requests.user,
                       temp_requests.super,
                       temp_requests.status,
                       temp_robot.uniqueid
                from temp_requests
                         left join temp_robot on temp_requests.phone_request = temp_robot.phone;
            """
        
        df = pd.DataFrame(client_click.query_dataframe(sql))
        
        df.fillna('unknown', inplace=True)
        df.replace(r'\N', 'unknown', inplace=True)

        df_to_file = f'{req_year}_{req_month}.csv'
        full_path = f'{path}{df_to_file}'
        df.to_csv(full_path, index=False, sep=';', encoding='utf-8')
        
        client_click.insert_dataframe('INSERT INTO suitecrm_robot_ch.all_requests_id VALUES',
                            df[['request_date', 'project', 'phone_request', 'user', 'super', 
                                'status', 'uniqueid']])

        time_2 = time.time()
#         print(sql)
        print(df.head())
        print(f'Обработан: {df_to_file}')
        print(f'Ушло времени: {round(time_2 - time_1, 3)} сек.')
        print('-' * 70)

end_time = time.time()
print()
print(f'Ушло времени на все файлы: {round(end_time - start_time, 3)} сек.')


  request_date  project phone_request                                  user  \
0   2017-01-29  Beeline   89503142298  5fe83d59-adb5-6dc0-1053-57f4cc54ded2   
1   2017-01-11  Beeline   89503211360  1244575b-d2ad-3743-b0aa-5818a0772862   
2   2017-01-30  Beeline   89503272587  15131e32-ffaf-7060-ffbf-5881e2e04e7a   
3   2017-01-30  Beeline   89503278877  1911a5cf-ec45-6dc0-00c9-5829808b2b78   
4   2017-01-11  Beeline   89503402522  5d249700-faf3-e2b5-2996-57f67bf9ed91   

     super             status uniqueid  
0  unknown  provider_not_held  unknown  
1  unknown             Active  unknown  
2  unknown             Active  unknown  
3  unknown             Active  unknown  
4  unknown             Active  unknown  
Обработан: 2017_1.csv
Ушло времени: 1.333 сек.
----------------------------------------------------------------------
  request_date  project phone_request                                  user  \
0   2017-02-09  Beeline   89213577679  566db9a6-5ff5-108a-9351-5784cd4a2014   
1  

  request_date  project phone_request                                  user  \
0   2017-11-08      TTK   89140021656  a35200af-5ab4-3d00-f07c-591ed8c50101   
1   2017-11-21  Beeline   89060100785  2a3f19c4-902f-0002-1d65-599eca1cc002   
2   2017-11-01  Beeline   89060365807  8acce750-0ea6-d9c4-576f-599ecb85b4a4   
3   2017-11-13  Beeline   89060374974  27d4ae7e-737a-5ec1-e5b3-5a05c38cd16d   
4   2017-11-17  Beeline   89060393033  7e9d6493-0743-2f0b-a761-59dba2191e7a   

     super           status uniqueid  
0  unknown          Created  unknown  
1  unknown  provider_otmena  unknown  
2  unknown  provider_otmena  unknown  
3  unknown  provider_otmena  unknown  
4  unknown           Active  unknown  
Обработан: 2017_11.csv
Ушло времени: 1.045 сек.
----------------------------------------------------------------------
  request_date  project phone_request                                  user  \
0   2017-12-25  Beeline   89506900643  30c8ffd2-de48-6a40-17a5-582c7acaa615   
1   2017-12-15

  request_date  project phone_request                                  user  \
0   2018-09-12      TTK   89137607966  a260acaf-e05d-1e59-f888-5b2103f5e264   
1   2018-09-17  Beeline             8  3f220471-c01e-775d-2211-5b069b21cc2d   
2   2018-09-05  Beeline             8  7893a5e8-5038-cdb5-9059-5b069ccca406   
3   2018-09-05  Beeline             8  ecdbc7e5-b622-854e-a431-5a295e138d7f   
4   2018-09-21  Beeline             8  c91dbc69-6b64-92b1-06c1-5b0697bbbd2b   

                                  super           status uniqueid  
0  2e571efb-5db6-779b-f91e-58f9bd475569  provider_otmena  unknown  
1  971d077d-e9c4-06de-a181-5811d2b8bef5          Created  unknown  
2  971d077d-e9c4-06de-a181-5811d2b8bef5          Created  unknown  
3                               unknown          unknown  unknown  
4  971d077d-e9c4-06de-a181-5811d2b8bef5          Created  unknown  
Обработан: 2018_9.csv
Ушло времени: 1.14 сек.
----------------------------------------------------------------------


  request_date  project phone_request                                  user  \
0   2019-06-29  Beeline   89213881200  73486ba0-3024-0dff-4371-5c18b419bcc3   
1   2019-06-05  Beeline   89213982050  7867246c-2313-484a-3cff-57eaa3476f1b   
2   2019-06-13  Beeline   89214203732  577b48e8-da66-b21a-43b0-5cae009db0bc   
3   2019-06-05  Beeline   89214287719  105a6690-444b-a086-0045-5c8b4a3bfe29   
4   2019-06-05  Beeline   89214816353  7867246c-2313-484a-3cff-57eaa3476f1b   

                                  super           status uniqueid  
0  91c8f3e4-78f2-4a36-99d0-5a846bcd4741           Active  unknown  
1  df1529db-4c5e-9d5e-1c1a-592d49adeff7           Active  unknown  
2  3238e441-57fc-910a-4ed7-57e251b88d76           Active  unknown  
3  91c8f3e4-78f2-4a36-99d0-5a846bcd4741  provider_otmena  unknown  
4  df1529db-4c5e-9d5e-1c1a-592d49adeff7           Active  unknown  
Обработан: 2019_6.csv
Ушло времени: 1.393 сек.
----------------------------------------------------------------------

  request_date  project phone_request                                  user  \
0   2020-03-16      RTK   89637105527  f3773f5b-15b5-3afd-ffcb-5c7cf4f3931e   
1   2020-03-15  Beeline   89637117765  7867246c-2313-484a-3cff-57eaa3476f1b   
2   2020-03-19      RTK   89637121119  317f76ae-9c27-dbae-8459-5e55141c26b0   
3   2020-03-09      RTK   89637225671  b08501d4-a636-060c-9daf-5dad5f69fbd8   
4   2020-03-19  Beeline   89637262922  779c9f3d-e055-c0ba-f6e6-5e3be488f07f   

                                  super           status uniqueid  
0  ab18474b-994d-4571-997b-5b59a6cdb507  provider_otmena  unknown  
1  9e9144d6-8554-addd-8079-5ba207b239be  provider_otmena  unknown  
2  83dc9df4-d2d2-a427-f1f0-5acf04021b53           Active  unknown  
3  2f24822c-0d50-df1e-e5cd-5c49d78502e9  provider_otmena  unknown  
4  12cfe0f5-ed86-48cd-fa31-5de4c929ff03  provider_otmena  unknown  
Обработан: 2020_3.csv
Ушло времени: 1.789 сек.
----------------------------------------------------------------------

  request_date project phone_request                                  user  \
0   2020-12-09   DOMRU   89213529093  965f6374-bc83-de0f-353c-5ce79f20cc95   
1   2020-12-05   DOMRU   89213543620  cd95d711-f56f-33d0-48ca-5efedeb1f0d9   
2   2020-12-08     RTK   89213577567  42aec8c8-8caf-29a9-e83e-5f68c7862605   
3   2020-12-07   DOMRU   89213720842  9195a8fa-08ed-6f0e-69f4-5d515a6948b1   
4   2020-12-04     RTK   89213769277  32bf1e3e-20b6-37a2-fb05-5f8ef62a697b   

                                  super           status uniqueid  
0  1957fd05-5d1e-6253-652a-5c6676885bf0             Held  unknown  
1  1957fd05-5d1e-6253-652a-5c6676885bf0             Held  unknown  
2  e548401c-d7f0-d8ee-7e62-58492c4e06cb  provider_otmena  unknown  
3  1957fd05-5d1e-6253-652a-5c6676885bf0  provider_otmena  unknown  
4  a78410f0-96d4-e2cb-25d0-5f8ef44244d3  provider_otmena  unknown  
Обработан: 2020_12.csv
Ушло времени: 1.487 сек.
----------------------------------------------------------------------
  re

  request_date  project phone_request                                  user  \
0   2021-09-29    DOMRU   89213533211  622fe943-7a94-d790-d04b-601bea680927   
1   2021-09-06  Beeline   89213828335  7d8c43d3-5d0c-ccf0-4381-6107ffe46bfb   
2   2021-09-27    DOMRU   89213837132  622fe943-7a94-d790-d04b-601bea680927   
3   2021-09-24      NBN   89213890607  ccf75c40-5b52-743e-e4e5-6138be7115c4   
4   2021-09-16      MTS   89214245262  d2815619-7daa-bc2e-9290-5e5cf3f9f1d7   

                                  super           status         uniqueid  
0  4995f87c-1857-f50d-e7a7-60191b495c64           Active          unknown  
1  2aff7afe-97eb-9c02-2e26-5d35c9057af2  provider_otmena          unknown  
2  4995f87c-1857-f50d-e7a7-60191b495c64          Created          unknown  
3  ab18474b-994d-4571-997b-5b59a6cdb507         Declined  1625407454.5397  
4  a67ee6c3-9a38-f545-cf82-5b192412bb71           Active          unknown  
Обработан: 2021_9.csv
Ушло времени: 42.289 сек.
---------------------

  request_date project phone_request                                  user  \
0   2022-05-07     RTK   89059614690  bad58214-ff5c-7ba9-3828-61641d3468d0   
1   2022-05-13     MTS   89059614807  ba5974af-982f-0031-e4f7-60dacbfb94ac   
2   2022-05-15     RTK   89059620741  2014315f-74c7-e279-6aaf-624fdf697456   
3   2022-05-16     TTK   89059644313  39a42527-478e-b33b-e56e-6210ea6ea6f1   
4   2022-05-31     MTS   89059650490  8451cf9c-b3a1-d4e1-6385-62909768a0f0   

                                  super           status              uniqueid  
0  debbb754-202e-3e6c-673d-60af871fbbc8  provider_otmena  1651917404.3712487.7  
1  d3f00f27-631c-d4e1-27ea-60bb4b460141  provider_otmena  1652423339.8997628.5  
2  debbb754-202e-3e6c-673d-60af871fbbc8           Active    1652608082.3308.22  
3  ae983650-afef-b9c2-88a1-6013c7691a72           Active    1652691882.5616.17  
4  7805f6dc-a93f-a404-99a4-615f0e727d4e             Held  1654010437.3539763.6  
Обработан: 2022_5.csv
Ушло времени: 167.425 с

  request_date project phone_request                                  user  \
0   2023-01-13     MTS   89213599414  9dba5d58-34cc-d9c9-7a38-6012b9166f50   
1   2023-01-22     RTK   89213599929  787f5f58-7682-5a95-c5c3-631c580c8032   
2   2023-01-21     RTK   89213600311  4b56e9fe-faf9-62e4-226e-63903b7f9885   
3   2023-01-08     RTK   89213619098  cbff0187-feb1-b9a6-e838-63a2d7774e59   
4   2023-01-20     TTK   89213642735  3ff4f626-76fc-cd0b-ffe9-639acac6a68f   

                                  super            status  \
0  3b8557d3-8242-1748-e120-5bbb59808f60              Held   
1  d24daf6d-4255-0fb5-1766-6310b1e811b0  provider_planned   
2  87d8e952-69a1-f209-2ebf-6310b5e5b67f  provider_planned   
3  6c41d121-416d-60d7-9cf0-6310ade462e1              Held   
4  ae983650-afef-b9c2-88a1-6013c7691a72   provider_otmena   

               uniqueid  
0    1673609275.86486.6  
1   1674378792.27747.28  
2   1674305996.97689.28  
3  1674660800.149796.26  
4   1674205830.47787.26  
Обработа